In [15]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import json
from IPython import display

from tqdm import tqdm, tqdm_notebook, tqdm_pandas
tqdm.pandas()

import copy

from Scripts.saving import submit

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from Scripts.utils import *

import json

import featuretools as ft

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
!tree {input_dir}

/home/furfa/work/ai-academy2019/input
├── ai-academy-2019-skill-prediction-data-csv-v1.zip
├── ai-academy-2019-skill-prediction-data-jsonlines.zip
├── dota2_abilities.csv
├── dota2_heroes.csv
├── dota2_items.csv
├── dota2_skill_test.csv
├── dota2_skill_test.jsonlines
├── dota2_skill_train.csv
├── dota2_skill_train.jsonlines
└── processed
    ├── heroes_proc.csv
    ├── items_proc.csv
    ├── test_all_JSON.csv
    ├── test_levelup.csv
    ├── test_proc.csv
    ├── test_team.csv
    ├── test_timeseries.csv
    ├── train_all_JSON.csv
    ├── train_levelup.csv
    ├── train_proc.csv
    ├── train_team.csv
    └── train_timeseries.csv

1 directory, 21 files


In [3]:
# aditional
abilities = pd.read_csv( os.path.join(input_dir, 'dota2_abilities.csv') ).drop(["dname", "desc"], axis=1)
items = pd.read_csv( os.path.join(input_dir,"processed", 'items_proc.csv') )
heroes = pd.read_csv( 
    os.path.join(input_dir,"processed", 'heroes_proc.csv') 
).set_index("hero_id").drop('Unnamed: 0', axis=1)

In [4]:
# Main dataset
train = pd.read_csv( os.path.join(input_dir,"processed", 'train_all_JSON.csv'), index_col=0 )
test = pd.read_csv( os.path.join(input_dir, "processed", 'test_all_JSON.csv'), index_col=0 )
json_columns = ['damage_targets', 
                'final_items', 
                'ability_upgrades',
                'series',
                'dire_heroes',
                'level_up_times',
                'item_purchase_log',
                'radiant_heroes'
               ]

In [5]:
print("Deleted", set(train.columns).difference( set(test.columns) ) )

y_train = train.skilled
train.drop(["skilled"], 1, inplace = True)

Deleted {'skilled'}


In [6]:
def pipe(data, type_ = "train"):
    def tr(x):
        if x=='dire':
            return 0
        elif x=='radiant':
            return 1
    data["player_team"] = data["player_team"].apply(tr)
    data["winner_team"] = data["winner_team"].apply(tr)
    data_json = data[json_columns]
    data.drop(json_columns, axis=1, inplace=True)
    
    train_timeser = pd.read_csv( os.path.join(input_dir, "processed", f"{type_}_timeseries.csv"), index_col=0 )
    train_timeser.drop(
        [col for col in train_timeser.columns if col.endswith("mode")],
        axis=1,
        inplace=True
    )
    data = pd.concat([data, train_timeser],axis=1)
    
    train_level = pd.read_csv( os.path.join(input_dir, "processed", f"{type_}_levelup.csv"), index_col=0 )
    data = pd.concat([data, train_level],axis=1)
    
    def prep_heroes(ser, team):
        new_data = {f"{team}{i}":list() for i in range(1,6)}
        for line in ser:
            line = json.loads(line)
            line.sort()
            for i in range(1,6):
                new_data[f"{team}{i}"].append(line[i-1])
        return pd.DataFrame(new_data, index=ser.index)
    data = pd.concat(
        [
            data,
            prep_heroes(data_json.radiant_heroes , "radiant" ),
            prep_heroes(data_json.dire_heroes , "dire" )
        ], axis=1
    )
    data_json.drop(["radiant_heroes", "dire_heroes"], axis=1, inplace=True)
    
    def prep_final_items(ser):
        new_data = {f"fin_item_{i}":list() for i in range(1,10)}
        for line in ser:
            line = json.loads(line)
            for i in range(1,10):
                new_data[f"fin_item_{i}"].append(line[i-1])
        return pd.DataFrame(new_data, index=ser.index)
    
    data = pd.concat(
        [
            data,
            prep_final_items(data_json.final_items),
        ], axis=1
    )
    data_json.drop("final_items", axis=1, inplace=True)
    
    return data

In [7]:
train = pipe(train, "train")

In [8]:
test = pipe(test, "test")

### Суммы по тому - какие возможности абилки качал

In [13]:
from sklearn.model_selection import cross_val_score, cross_val_predict
import catboost as cb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    train,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1,
    n_jobs=-1
)

Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.0min finished


array([0.80318808, 0.80386895, 0.80330823, 0.80378099])

In [14]:
model = cb.CatBoostClassifier(logging_level="Silent").fit(train, y_train)
y_pred = model.predict(test)
y_pred = pd.DataFrame({'id' : test.index, 'skilled': y_pred})
submit(y_pred, "cb_conveer_raw")

Learning rate set to 0.061655
File saved in : /home/furfa/work/ai-academy2019/predictions/cb_conveer_raw[2019-03-30 02:03:05.910488].csv


## Оставить в матчах только уникальные
unique_index = dict()
for ind in tqdm_notebook(data_match.index):
    selection = data_match
    
    for col in data_match.columns:
        selection = selection[ selection[col] == data_match.ix[ind, col] ]
    
    ind_list = selection.index
    unique_index[ind] = ind_list[0]

In [43]:
data = reduce_mem_usage(data)

Memory usage of dataframe is 132.58 MB
Memory usage after optimization is: 37.91 MB
Decreased by 71.4%


In [17]:
var_types = {
    name : p[0] for p in ( 
                            (ft.variable_types.Categorical, 
                             [
                                "dire_barracks_status", "dire_tower_status",
                                "radiant_tower_status", "radiant_barracks_status",
                                "radiant1","radiant2","radiant3","radiant4","radiant5",
                                "dire1","dire2","dire3","dire4", "dire5", "party_players", "roshan_kills", "hero_id",
                                 "fin_item_1","fin_item_2","fin_item_3","fin_item_4","fin_item_5",
                                 "fin_item_6","fin_item_7","fin_item_8","fin_item_9"
                             ]),
                            (ft.variable_types.Boolean, 
                             [
                                "winner_team", "first_blood_claimed",
                                 "leaver_status", "player_team"
                             ]),
                         ) for name in p[1]
}


In [16]:
def generate_features(data, var_types, trans_primitives=["multiply",'divide', "diff"], N_FEATURES=1000):
    
    start_columns = data.columns
    
    data = data.reset_index()
    data.id = data.id.astype(np.int32)
    
    N_FEATURES += data.shape[1]
    
    es = ft.EntitySet(id='players')
    
    main_entity_id = 'train_players'

    df = data.copy()
    # Entities with a unique index
    es = es.entity_from_dataframe(
        entity_id=main_entity_id, 
        dataframe=df, # dataframe object
        index='id', # unique index
        variable_types=var_types
    )

    print(es)
    
    # DFS with specified primitives
    features, feature_names = ft.dfs(
        entityset=es, 
        target_entity=main_entity_id,
        trans_primitives = trans_primitives,
        agg_primitives=[], 
        max_depth=1, 
        features_only=False,
        verbose=True,
        chunk_size=0.5,
        max_features=N_FEATURES, # comment it later, computational burden reduction
         n_jobs=-1,
    )
    return features.drop(start_columns, axis=1)

In [26]:
multi_features = generate_features(train, 
                                   var_types,
                                   trans_primitives=["multiply"],
                                   N_FEATURES=1000)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 174]
  Relationships:
    No relationships


Exception ignored in: <generator object add_client at 0x7f5e022e8620>
RuntimeError: generator ignored GeneratorExit
Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/ana

Built 1174 features
EntitySet scattered to workers in 1.442 seconds

Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/3 chunks
Elapsed: 00:01 | Remaining: 00:02 | Progress:  33%|███▎      | Calculated: 1/3 chunks
Elapsed: 00:14 | Remaining: 00:04 | Progress:  67%|██████▋   | Calculated: 2/3 chunks
Elapsed: 00:15 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


In [27]:
divide_features = generate_features(train, 
                                   var_types,
                                   trans_primitives=['divide'],
                                   N_FEATURES=1000)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 174]
  Relationships:
    No relationships
Built 1174 features
EntitySet scattered to workers in 1.487 seconds

Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/3 chunks
Elapsed: 00:01 | Remaining: 00:02 | Progress:  33%|███▎      | Calculated: 1/3 chunks
Elapsed: 00:12 | Remaining: 00:04 | Progress:  67%|██████▋   | Calculated: 2/3 chunks
Elapsed: 00:13 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


In [ ]:
def get_most_importance(df, y_train):
    model = cb.CatBoostClassifier(logging_level="Silent").fit(df, y_train)
    imp = pd.DataFrame({
        "name": df.columns,
        "importance" : model.feature_importances_,
    })
    imp = imp.sort_values(by="importance", ascending=False)
    
    print(
        cross_val_score(
            model,
            df,
            y_train,
            scoring="accuracy",
            cv = 4,
            verbose=0,
            n_jobs=-1
        )
    )
    
    return imp

print("multi")
multi_imp = get_most_importance(multi_features, y_train)
print("divide")
divide_imp = get_most_importance(divide_features, y_train)

multi
Learning rate set to 0.061655


Future exception was never retrieved
future: <Future finished exception=StreamClosedError('Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/tcpclient.py", line 112, in on_connect_done
    stream = future.result()
tornado.iostream.StreamClosedError: Stream is closed


Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834


Process ForkPoolWorker-42:
Process ForkPoolWorker-52:
Process ForkPoolWorker-50:
Process ForkPoolWorker-46:
Process ForkPoolWorker-45:
Process ForkPoolWorker-48:
Process ForkPoolWorker-53:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-51:
Process ForkPoolWorker-44:
  File "/home/furfa/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-43:
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self.

In [53]:
cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    features,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1,
    n_jobs=-1
)

Exception ignored in: <generator object add_client at 0x7f48579a6410>
RuntimeError: generator ignored GeneratorExit


Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  6.5min finished


array([0.80026434, 0.79990388, 0.7989827 , 0.80201866])

In [61]:
cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    features[
        imp[:200].name.values
    ],
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1,
    n_jobs=-1
)

Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.1min finished


array([0.80358859, 0.80454982, 0.80146588, 0.80510274])

In [69]:
lp = ft.primitives.list_primitives()
lp[lp.type == 'transform']

,name,type,description
19,longitude,transform,Returns the second value on the tuple base fea...
20,is_null,transform,"For each value of base feature, return 'True' ..."
21,cum_sum,transform,Calculates the sum of previous values of an in...
22,seconds,transform,Transform a Timedelta feature into the number ...
23,year,transform,Transform a Datetime feature into the year.
24,percentile,transform,"For each value of the base feature, determines..."
25,month,transform,Transform a Datetime feature into the month.
26,mod,transform,Creates a transform feature that divides two f...
27,diff,transform,Compute the difference between the value of a ...
28,second,transform,Transform a Datetime feature into the second.


In [78]:
lp[lp.type == 'aggregation']

,name,type,description
0,time_since_last,aggregation,Time since last related instance.
1,avg_time_between,aggregation,Computes the average time between consecutive ...
2,max,aggregation,Finds the maximum non-null value of a numeric ...
3,all,aggregation,Test if all values are 'True'.
4,n_most_common,aggregation,Finds the N most common elements in a categori...
5,percent_true,aggregation,Finds the percent of 'True' values in a boolea...
6,min,aggregation,Finds the minimum non-null value of a numeric ...
7,any,aggregation,Test if any value is 'True'.
8,mode,aggregation,Finds the most common element in a categorical...
9,last,aggregation,Returns the last value.


## Фича : на каком месте в списке героев